## Libraries

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from loguru import logger

from nam.config import defaults
from nam.types import Config
from nam.utils.args import parse_args
from nam.data import NAMDataset
from nam.models import DNN, FeatureNN, NAM, get_num_units
from nam.engine import Engine

from main import get_config

import pytorch_lightning as pl

## Configurations

In [3]:
config = get_config()
pl.seed_everything(config.seed)

1

In [4]:
config

namespace(device='cpu',
          output_dir='output',
          training_epochs=10,
          lr=0.01,
          batch_size=1024,
          dropout=0.5,
          feature_dropout=0.0,
          decay_rate=0.995,
          l2_regularization=0.0,
          output_regularization=0.0,
          num_basis_functions=1000,
          units_multiplier=2,
          num_units=64,
          data_split=1,
          seed=1,
          cross_val=False,
          n_models=1,
          num_splits=3,
          fold_num=1,
          activation='exu',
          shuffle=True,
          regression=False,
          debug=False,
          shallow=False,
          use_dnn=False,
          patience=10,
          n_folds=5,
          num_workers=16,
          learning_rate=0.01,
          dataset_name='Teleco',
          max_checkpoints_to_keep=1,
          save_checkpoint_every_n_epochs=10,
          early_stopping_epochs=60)

## Dataset

In [5]:
features_columns = ["income_2", "WP1219", "WP1220", "weo_gdpc_con_ppp"]
targets_column = ["WP16"]
weights_column = ["wgt"]

In [6]:
data = pd.read_csv('data/GALLUP.csv')

In [7]:
missing = data.isnull().sum()
print(missing)

WP16                    0
wgt                     0
country                 0
income_2                0
WP1219                  1
WP1220                185
year                    0
weo_gdpc_con_ppp    24122
dtype: int64


In [8]:
data = data.fillna(method='ffill')

In [9]:
dataset = NAMDataset(config=config,
                    csv_file=data, #'data/GALLUP.csv',
                    features_columns=features_columns,
                    targets_column=targets_column,
                    weights_column=weights_column, one_hot=False)
dataset

In [10]:
dataset[0]

(tensor([1.8043e+04, 1.0000e+00, 1.5000e+01, 3.2658e+04]), tensor([4.]))

In [11]:
train_dataset, test_dataset = torch.utils.data.random_split(
  dataset,
  [int(np.floor(len(dataset) * .9)),
   int(np.ceil(len(dataset) * .1))],
)
len(train_dataset), len(test_dataset)

(1206728, 134081)

In [12]:
train_dataset, val_dataset = torch.utils.data.random_split(
  train_dataset,
  [int(np.floor(len(train_dataset) * .8)),
   int(np.ceil(len(train_dataset) * .2))],
)
len(train_dataset), len(val_dataset)

(965382, 241346)

In [13]:
train_dataloader = DataLoader(
  train_dataset,
  batch_size=config.batch_size,
  shuffle=True,
  num_workers=config.num_workers,
)
val_dataloader = DataLoader(
  val_dataset,
  batch_size=config.batch_size * 5,
  shuffle=False,
  num_workers=config.num_workers,
)

## NAM Model

In [14]:
model = NAM(
      config=config,
      name="NAMModel",
      num_inputs=len(dataset[0][0]),
      num_units=get_num_units(config, train_dataloader),
).to(device=config.device)

/Users/amrmkayid/anaconda3/envs/nam/lib/python3.7/site-packages/torch/nn/init.py:162: UserWarning: mean is more than 2 std from [a, b] in nn.init.trunc_normal_. The distribution of values may be incorrect.
  return _no_grad_trunc_normal_(tensor, mean, std, a, b)


In [15]:
model

NAM(
  (feature_nns): Sequential(
    (FeatureNN_0): FeatureNN(
      (relu): ReLU(inplace=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (model): Sequential(
        (0): ExU(in_features=1, out_features=1000)
        (1): Linear(in_features=1000, out_features=64, bias=True)
        (2): Linear(in_features=64, out_features=32, bias=True)
        (3): Linear(in_features=32, out_features=1, bias=True)
      )
    )
    (FeatureNN_1): FeatureNN(
      (relu): ReLU(inplace=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (model): Sequential(
        (0): ExU(in_features=1, out_features=4)
        (1): Linear(in_features=4, out_features=64, bias=True)
        (2): Linear(in_features=64, out_features=32, bias=True)
        (3): Linear(in_features=32, out_features=1, bias=True)
      )
    )
    (FeatureNN_2): FeatureNN(
      (relu): ReLU(inplace=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (model): Sequential(
        (0): ExU(in_features=1, out_features=

## Training

In [16]:
engine = Engine(config, model)

In [17]:
early_stop_callback = pl.callbacks.early_stopping.EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=config.patience,
   verbose=False,
   mode='max'
)

In [18]:
trainer = pl.Trainer(default_root_dir=config.output_dir, 
                     max_epochs=config.training_epochs, 
                     callbacks=[early_stop_callback])

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [19]:
trainer.fit(engine, train_dataloader, val_dataloader)


  | Name  | Type | Params
-------------------------------
0 | model | NAM  | 148 K 
-------------------------------
148 K     Trainable params
0         Non-trainable params
148 K     Total params


Validation sanity check: 100%|██████████| 2/2 [00:00<00:00,  3.00it/s]

/Users/amrmkayid/anaconda3/envs/nam/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0:  95%|█████████▌| 943/991 [00:41<00:02, 22.61it/s, loss=-3.11, v_num=0, val_loss_epoch=0.693, train_loss_step=-3.16]
Validating: 0it [00:00, ?it/s]
Epoch 1:  95%|█████████▌| 943/991 [00:39<00:02, 23.66it/s, loss=-3.45, v_num=0, val_loss_epoch=-4.21, train_loss_step=-3.69, val_loss_step=-4.17, train_loss_epoch=-3.4]
Validating: 0it [00:00, ?it/s]
Epoch 2:  95%|█████████▌| 943/991 [00:42<00:02, 22.00it/s, loss=-3.68, v_num=0, val_loss_epoch=-4.21, train_loss_step=-3.72, val_loss_step=-4.17, train_loss_epoch=-3.32]
Validating: 0it [00:00, ?it/s]
Epoch 3:  95%|█████████▌| 943/991 [00:37<00:01, 25.26it/s, loss=-3.24, v_num=0, val_loss_epoch=-4.21, train_loss_step=-3.05, val_loss_step=-4.17, train_loss_epoch=-3.1] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  95%|█████████▌| 944/991 [00:37<00:01, 24.90it/s, loss=-3.2, v_num=0, val_loss_epoch=-4.21, train_loss_step=-3.04, val_loss_step=-4.17, train_loss_epoch=-3.29] 
Validating: 0it [00:00, ?it/s]
Epoch 5:  95%|█████████▌| 944/991 [00:36<

1

## Testing

In [20]:
test_datalloader = DataLoader(
  test_dataset,
  batch_size=config.batch_size,
  shuffle=False,
  num_workers=config.num_workers,
)

In [21]:
trainer.test(engine, test_datalloader)

Testing: 100%|██████████| 131/131 [00:01<00:00, 75.47it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': tensor(-4.1810),
 'test_loss_epoch': tensor(-4.2002),
 'val_loss': tensor(-4.1669),
 'val_loss_epoch': tensor(-4.2112)}
--------------------------------------------------------------------------------


[{'val_loss_epoch': -4.21120023727417,
  'val_loss': -4.166907787322998,
  'test_loss_epoch': -4.200181007385254,
  'test_loss': -4.181014537811279}]

### TODO: OLD K-fold

## GALLUP Data

In [16]:
features_columns = ["income_2", "WP1219", "WP1220", "weo_gdpc_con_ppp"]
targets_column = ["WP16"]
weights_column = ["wgt"]

In [17]:
data = pd.read_csv('data/GALLUP.csv')
# data = data[:1000]

In [18]:
data = pd.read_csv('data/GALLUP.csv')
# data = data[:1000]
missing = data.isnull().sum()
print(missing)

WP16                    0
wgt                     0
country                 0
income_2                0
WP1219                  1
WP1220                185
year                    0
weo_gdpc_con_ppp    24122
dtype: int64


In [22]:
data = data.fillna(method='ffill')

In [23]:
dataset = NAMDataset(config=config,
                    csv_file=data, #'data/GALLUP.csv',
                    features_columns=features_columns,
                    targets_column=targets_column,
                    weights_column=weights_column, one_hot=False)
dataset

In [59]:
data_loaders = dataset.data_loaders(
      n_splits=config.num_splits,
      batch_size=config.batch_size,
      shuffle=config.shuffle,
      stratified=not config.regression,
      random_state=config.seed,
)

In [8]:
for train_dl, val_dl in dataset.data_loaders():
    print(next(iter(train_dl)), next(iter(val_dl)))

Fold((1,)), train: 938566, test: 134081
[tensor([[76845,     1,    40,   605],
        [80149,     0,    43,   861],
        [    0,     1,     4,   677],
        ...,
        [99347,     1,    44,   965],
        [79297,     0,    46,   940],
        [85594,     0,    24,   915]]), tensor([[  8883],
        [106160],
        [146743],
        ...,
        [ 12864],
        [ 77549],
        [  6516]]), tensor([[  8],
        [100],
        [ 62],
        ...,
        [ 50],
        [135],
        [ 68]])] [tensor([[ 30686,      0,     31,    389],
        [103458,      1,     47,   1040],
        [ 41558,      0,      5,    133],
        ...,
        [ 71748,      1,      6,    442],
        [ 75880,      0,     48,    965],
        [ 65557,      1,     20,    758]]), tensor([[ 54025],
        [233171],
        [ 57097],
        ...,
        [105269],
        [ 35618],
        [251079]]), tensor([[ 61],
        [133],
        [119],
        ...,
        [ 17],
        [ 50],
        [

In [12]:
batch = next(iter(train_dl))
batch[0].shape

torch.Size([1024, 4])

## Housing Data

In [ ]:
features_columns = [
    "longitude", "latitude", "housing_median_age", "total_rooms",
     "population", "households", "median_income", #"total_bedrooms",
]
targets_column = ["median_house_value"]
dataset = NAMDataset(config=config,
                    csv_file='data/housing.csv',
                    features_columns=features_columns,
                    targets_column=targets_column, 
                    one_hot=False)
len(dataset)

In [20]:

features_columns = [
    "longitude", "latitude", "housing_median_age", "total_rooms",
    "total_bedrooms", "population", "households", "median_income"
]
targets_column = ["median_house_value"]

In [22]:
dataset = NAMDataset(config=config,
                    csv_file='data/housing.csv',
                    features_columns=features_columns,
                    targets_column=targets_column, one_hot=False)
dataset

In [6]:
for train_dl, test_dl in dataset.data_loaders():
    print(next(iter(train_dl)), next(iter(test_dl)))

Fold((1,)), train: 16512, test: 4128
[tensor([[ 252,  583,   16,  ..., 2395, 1314, 9240],
        [ 484,  350,   33,  ...,  753,  296, 5622],
        [ 573,  124,   45,  ..., 1518,  586, 1051],
        ...,
        [ 330,  468,   26,  ..., 1435,  478, 3444],
        [ 201,  472,    9,  ..., 2002, 1000, 5214],
        [ 271,  570,   19,  ...,  907,  335, 8112]]), tensor([[1379],
        [ 328],
        [1334],
        ...,
        [ 276],
        [2098],
        [1775]])] [tensor([[  689,     4,    19,  ...,  1470,   352,  6979],
        [  677,    19,    51,  ...,   846,   533,  1960],
        [  631,   110,     1,  ...,  2820,   789, 12705],
        ...,
        [  563,   146,    39,  ...,   832,   229,   252],
        [  178,   468,    14,  ...,   952,   363, 12756],
        [  607,   120,     8,  ...,  1230,   164,  5482]]), tensor([[ 905],
        [ 710],
        [3786],
        ...,
        [ 930],
        [3841],
        [1237]])]
Fold((2,)), train: 16512, test: 4128
[tensor([[  

In [7]:
batch = next(iter(train_dl))
batch

[tensor([[  529,   165,    32,  ...,  1396,   469, 10207],
         [  684,    36,    15,  ...,   787,   290, 11128],
         [  185,   495,    25,  ...,  1374,   606,  3480],
         ...,
         [  595,   119,    21,  ...,  2351,   659, 11533],
         [  285,   616,    40,  ...,  1653,   709,  3520],
         [  463,   361,    26,  ...,  1151,   308,   555]]),
 tensor([[2064],
         [2127],
         [1425],
         ...,
         [2413],
         [ 974],
         [  75]])]

In [42]:
inputs = batch[0]
torch.chunk(inputs, 8, dim=-1)[0].shape

torch.Size([1024, 1])

In [43]:
inputs_tuple = torch.chunk(inputs, 8, dim=-1)
ind_out = [input_i for i, input_i in enumerate(inputs_tuple)]

In [50]:
stacked_out = torch.stack(ind_out, dim=-1)

In [53]:
torch.sum(stacked_out, dim=-1).shape

torch.Size([1024, 1])

In [9]:
batch[0].shape[-1], batch[1].shape

(8, torch.Size([1024, 1]))

In [20]:
nam = NAM(config=config, name="NAMModel", num_inputs=batch[0].shape[-1], num_units=64, shallow=False)

In [21]:
nam.feature_nns

[FeatureNN(
   (relu): ReLU(inplace=True)
   (dropout): Dropout(p=0.0, inplace=False)
   (_h1): Linear(in_features=64, out_features=64, bias=True)
   (_h2): Linear(in_features=64, out_features=32, bias=True)
   (linear): Linear(in_features=32, out_features=1, bias=True)
   (hidden_layers): Sequential(
     (0): ExU()
     (1): Linear(in_features=64, out_features=64, bias=True)
     (2): Linear(in_features=64, out_features=32, bias=True)
   )
 ),
 FeatureNN(
   (relu): ReLU(inplace=True)
   (dropout): Dropout(p=0.0, inplace=False)
   (_h1): Linear(in_features=64, out_features=64, bias=True)
   (_h2): Linear(in_features=64, out_features=32, bias=True)
   (linear): Linear(in_features=32, out_features=1, bias=True)
   (hidden_layers): Sequential(
     (0): ExU()
     (1): Linear(in_features=64, out_features=64, bias=True)
     (2): Linear(in_features=64, out_features=32, bias=True)
   )
 ),
 FeatureNN(
   (relu): ReLU(inplace=True)
   (dropout): Dropout(p=0.0, inplace=False)
   (_h1): Line

In [19]:
nam(batch[0])

tensor([-7615.4722, -8860.6367, -2092.2812,  ..., -8710.9600, -1141.4934,
          300.8806], grad_fn=<AddBackward0>)

In [9]:
fnn = FeatureNN(config=config, name="Feature_NN", input_shape=batch[0].shape, num_units=64, shallow=False)

In [10]:
fnn

FeatureNN(
  (relu): ReLU(inplace=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (_h1): Linear(in_features=64, out_features=64, bias=True)
  (_h2): Linear(in_features=64, out_features=32, bias=True)
  (linear): Linear(in_features=32, out_features=1, bias=True)
  (hidden_layers): Sequential(
    (0): ExU()
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=32, bias=True)
  )
)

In [14]:
fnn(batch[0])

tensor([-20471.6133,  -8078.4175, -13975.0645,  ...,  37003.4414,
          7193.8286,   9193.8447], grad_fn=<SqueezeBackward1>)